# how NN to working

### updating and RMSE

In [2]:

import tensorflow as tf

tf.set_random_seed(777)  # reproducibility

# tf Graph Input
x_data = [[1.],
          [2.],
          [3.]]
y_data = [[1.],
          [2.],
          [3.]]


# placeholders for a tensor that will be always fed.
X = tf.placeholder(tf.float32, shape=[None, 1])
Y = tf.placeholder(tf.float32, shape=[None, 1])

# Set wrong model weights
W = tf.Variable(tf.truncated_normal([1, 1]))
b = tf.Variable(5.)

# Forward prop
hypothesis = tf.matmul(X, W) + b

# diff
assert hypothesis.shape.as_list() == Y.shape.as_list()
diff = (hypothesis - Y)

# Back prop (chain rule)
d_l1 = diff
d_b = d_l1
d_w = tf.matmul(tf.transpose(X), d_l1)

print(X, W, d_l1, d_w)

# Updating network using gradients
learning_rate = 0.1
step = [
    tf.assign(W, W - learning_rate * d_w),
    tf.assign(b, b - learning_rate * tf.reduce_mean(d_b)),
]

# 7. Running and testing the training process
RMSE = tf.reduce_mean(tf.square((Y - hypothesis)))

sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

for i in range(1000):
    print(i, sess.run([step, RMSE], feed_dict={X: x_data, Y: y_data}))

print(sess.run(hypothesis, feed_dict={X: x_data}))

Tensor("Placeholder_2:0", shape=(?, 1), dtype=float32) Tensor("Variable_2/read:0", shape=(1, 1), dtype=float32) Tensor("sub_4:0", shape=(?, 1), dtype=float32) Tensor("MatMul_3:0", shape=(1, 1), dtype=float32)
0 [[array([[-1.47523117]], dtype=float32), 4.7623844], 6.7935429]
1 [[array([[-0.86733818]], dtype=float32), 4.7811923], 4.1198864]
2 [[array([[-1.12178016]], dtype=float32), 4.6765409], 3.4198303]
3 [[array([[-0.95721245]], dtype=float32), 4.6332426], 3.1887729]
4 [[array([[-0.9970606]], dtype=float32), 4.5613608], 3.0704861]
5 [[array([[-0.93799227]], dtype=float32), 4.5046368], 2.9805949]
6 [[array([[-0.92758512]], dtype=float32), 4.4417715], 2.8990796]
7 [[array([[-0.89402884]], dtype=float32), 4.3831115], 2.8211572]
8 [[array([[-0.87225533]], dtype=float32), 4.323606], 2.7456522]
9 [[array([[-0.84526145]], dtype=float32), 4.2656965], 2.6722448]
10 [[array([[-0.82131338]], dtype=float32), 4.208179], 2.6008177]
11 [[array([[-0.79638207]], dtype=float32), 4.1516237], 2.5313044]


In [2]:
"""
In this file, we will implement back propagations by hands
We will use the Sigmoid Cross Entropy loss function.
This is equivalent to tf.nn.sigmoid_softmax_with_logits(logits, labels)
[References]
1) Tensorflow Document (tf.nn.sigmoid_softmax_with_logits)
    https://www.tensorflow.org/api_docs/python/tf/nn/sigmoid_cross_entropy_with_logits
2) Neural Net Backprop in one slide! by Sung Kim
    https://docs.google.com/presentation/d/1_ZmtfEjLmhbuM_PqbDYMXXLAqeWN0HwuhcSKnUQZ6MM/edit#slide=id.g1ec1d04b5a_1_83
3) Back Propagation with Tensorflow by Dan Aloni
    http://blog.aloni.org/posts/backprop-with-tensorflow/
4) Yes you should understand backprop by Andrej Karpathy
    https://medium.com/@karpathy/yes-you-should-understand-backprop-e2f06eab496b#.cockptkn7
[Network Architecture]
Input: x
Layer1: x * W + b
Output layer = σ(Layer1)
Loss_i = - y * log(σ(Layer1)) - (1 - y) * log(1 - σ(Layer1))
Loss = tf.reduce_sum(Loss_i)
We want to compute that
dLoss/dW = ???
dLoss/db = ???
please read "Neural Net Backprop in one slide!" for deriving formulas
"""
import tensorflow as tf
import numpy as np
tf.set_random_seed(777)  # for reproducibility

# Predicting animal type based on various features
xy = np.loadtxt('data04.csv', delimiter=',', dtype=np.float32)
X_data = xy[:, 0:-1]
N = X_data.shape[0]
y_data = xy[:, [-1]]

# y_data has labels from 0 ~ 6
print("y has one of the following values")
print(np.unique(y_data))

# X_data.shape = (101, 16) => 101 samples, 16 features
# y_data.shape = (101, 1)  => 101 samples, 1 label
print("Shape of X data: ", X_data.shape)
print("Shape of y data: ", y_data.shape)

nb_classes = 7  # 0 ~ 6

X = tf.placeholder(tf.float32, [None, 16])
y = tf.placeholder(tf.int32, [None, 1])  # 0 ~ 6

target = tf.one_hot(y, nb_classes)  # one hot
target = tf.reshape(target, [-1, nb_classes])
target = tf.cast(target, tf.float32)

W = tf.Variable(tf.random_normal([16, nb_classes]), name='weight')
b = tf.Variable(tf.random_normal([nb_classes]), name='bias')


def sigma(x):
    # sigmoid function
    # σ(x) = 1 / (1 + exp(-x))
    return 1. / (1. + tf.exp(-x))


def sigma_prime(x):
    # derivative of the sigmoid function
    # σ'(x) = σ(x) * (1 - σ(x))
    return sigma(x) * (1. - sigma(x))


# Forward propagtion
layer_1 = tf.matmul(X, W) + b
y_pred = sigma(layer_1)

# Loss Function (end of forwad propagation)
loss_i = - target * tf.log(y_pred) - (1. - target) * tf.log(1. - y_pred)
loss = tf.reduce_sum(loss_i)

# Dimension Check
assert y_pred.shape.as_list() == target.shape.as_list()


# Back prop (chain rule)
# How to derive? please read "Neural Net Backprop in one slide!"
d_loss = (y_pred - target) / (y_pred * (1. - y_pred) + 1e-7)
d_sigma = sigma_prime(layer_1)
d_layer = d_loss * d_sigma
d_b = d_layer
d_W = tf.matmul(tf.transpose(X), d_layer)

# Updating network using gradients
learning_rate = 0.01
train_step = [
    tf.assign(W, W - learning_rate * d_W),
    tf.assign(b, b - learning_rate * tf.reduce_sum(d_b)),
]

# Prediction and Accuracy
prediction = tf.argmax(y_pred, 1)
acct_mat = tf.equal(tf.argmax(y_pred, 1), tf.argmax(target, 1))
acct_res = tf.reduce_mean(tf.cast(acct_mat, tf.float32))

# Launch graph
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for step in range(500):
        sess.run(train_step, feed_dict={X: X_data, y: y_data})

        if step % 10 == 0:
            # Within 300 steps, you should see an accuracy of 100%
            step_loss, acc = sess.run([loss, acct_res], feed_dict={
                                      X: X_data, y: y_data})
            print("Step: {:5}\t Loss: {:10.5f}\t Acc: {:.2%}" .format(
                step, step_loss, acc))

    # Let's see if we can predict
    pred = sess.run(prediction, feed_dict={X: X_data})
    for p, y in zip(pred, y_data):
        msg = "[{}]\t Prediction: {:d}\t True y: {:d}"
        print(msg.format(p == int(y[0]), p, int(y[0])))

y has one of the following values
[ 0.  1.  2.  3.  4.  5.  6.]
Shape of X data:  (101, 16)
Shape of y data:  (101, 1)
Step:     0	 Loss:  608.68427	 Acc: 37.62%
Step:    10	 Loss:  125.64272	 Acc: 70.30%
Step:    20	 Loss:   81.28768	 Acc: 92.08%
Step:    30	 Loss:   63.38433	 Acc: 95.05%
Step:    40	 Loss:   53.20226	 Acc: 95.05%
Step:    50	 Loss:   46.53529	 Acc: 96.04%
Step:    60	 Loss:   41.77147	 Acc: 96.04%
Step:    70	 Loss:   38.15804	 Acc: 96.04%
Step:    80	 Loss:   35.29751	 Acc: 98.02%
Step:    90	 Loss:   32.95963	 Acc: 99.01%
Step:   100	 Loss:   31.00125	 Acc: 99.01%
Step:   110	 Loss:   29.32831	 Acc: 99.01%
Step:   120	 Loss:   27.87626	 Acc: 99.01%
Step:   130	 Loss:   26.59926	 Acc: 99.01%
Step:   140	 Loss:   25.46377	 Acc: 99.01%
Step:   150	 Loss:   24.44465	 Acc: 100.00%
Step:   160	 Loss:   23.52268	 Acc: 100.00%
Step:   170	 Loss:   22.68285	 Acc: 100.00%
Step:   180	 Loss:   21.91329	 Acc: 100.00%
Step:   190	 Loss:   21.20444	 Acc: 100.00%
Step:   200	 Los